# Daten laden

---



In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/avocado-updated-2020.csv") # Patrick 07.10.2024: Daten im colab hochgeladen
df

FileNotFoundError: [Errno 2] No such file or directory: 'data/avocado-updated-2020.csv'

In [ ]:
# Prüfung, ob der Datenrahmen Nullwerte enthält
df.isnull().values.any()

In [ ]:
df.isnull().sum()

# Daten verstehen


---



In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.describe()

Brian

|   Spalte      | Typ     | Beschreibung                                    |
|---------------|---------|-------------------------------------------------|
| date          | object  |                                                 |
| average_price | float64 |                                                 |
| total_volume  | float64 |                                                 |
| 4046          | float64 | kleine/mittlere Bio-Hass-Avocados               |
| 4225          | float64 | große Hass-Avocados aus nicht                   |   
|               |         | biologischem Anbau                              |
| 4770          | float64 | nicht-biologische extra große Hass-Avocados     |
| total_bags    | float64 |                                                 |
| small_bags    | float64 |                                                 |
| large_bags    | float64 |                                                 |
| xlarge_bags   | float64 |                                                 |
| type          | object  |                                                 |
| year          | int64   |                                                 |
| geography     | object  |                                                 |


1.	Überblick über die Daten
  * 1.1.  4046: organic small/medium Hass Avocados (~3-5 oz)
  * 1.2.  4225: non-organic large Hass Avocados (~8-10 oz)
  * 1.2.  4770: non-organic extra large Hass Avocados (~10-15 oz)

* https://www.kaggle.com/code/shaniquehines/avocados-case-study

# Encoder

## DateEncoder date_number


In [ ]:
# Spalte 'date' in Datumsformat umwandeln (falls sie als String vorliegt)
df['date'] = pd.to_datetime(df['date'])

# Sortieren nach Datum (optional, um sicherzustellen, dass die Daten chronologisch sind)
df = df.sort_values(by='date').reset_index(drop=True)

# Berechnen des frühesten Datums im Datensatz
min_date = df['date'].min()

# Berechnen der Anzahl der Tage seit dem frühesten Datum
df['date_number'] = (df['date'] - min_date).dt.days

# Die Spalte 'date_number' an den Anfang verschieben
cols = ['date_number'] + [col for col in df.columns if col != 'date_number']  # Neue Reihenfolge der Spalten
df = df[cols]



## LabelEncoder type


In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
df['type'] = lb.fit_transform(df['type'])

In [ ]:
# Patrick LabelEncoder für geography


# Importieren der notwendigen Bibliotheken
from sklearn.preprocessing import LabelEncoder

# Initialisierung des LabelEncoders für die 'geography'-Spalte
geo_encoder = LabelEncoder()
# Anwenden des Encoders auf die 'geography'-Spalte
df['geography_encoded'] = geo_encoder.fit_transform(df['geography'])

# Anzeigen der ersten Zeilen des Datensatzes, um die Ergebnisse zu überprüfen
print(df[['geography', 'geography_encoded']].head())

# Optional: Speichern des modifizierten Datensatzes in eine neue CSV-Datei
# avocado_data.to_csv("data/avocado_encoded.csv", index=False)



## GeoDistanceEncoder

In [ ]:
# Patrick GeoDistanceEncoder für geography

# Importieren der notwendigen Bibliotheken
from geopy.distance import geodesic

# Definieren der Städte mit ihren Koordinaten
static_coordinates = [
    ('Albany', 42.6526, -73.7562),
    ('Atlanta', 33.749, -84.388),
    ('Baltimore/Washington', 39.2904, -76.6122),
    ('Boise', 43.615, -116.2023),
    ('Boston', 42.3601, -71.0589),
    ('Buffalo/Rochester', 43.1566, -77.6088),
    ('California', 36.7783, -119.4179),
    ('Charlotte', 35.2271, -80.8431),
    ('Chicago', 41.8781, -87.6298),
    ('Cincinnati/Dayton', 39.1031, -84.512),
    ('Columbus', 39.9612, -82.9988),
    ('Dallas/Ft. Worth', 32.7767, -96.797),
    ('Denver', 39.7392, -104.9903),
    ('Detroit', 42.3314, -83.0458),
    ('Grand Rapids', 42.9634, -85.6681),
    ('Great Lakes', 45.0, -84.0),
    ('Harrisburg/Scranton', 40.2732, -76.8867),
    ('Hartford/Springfield', 41.7627, -72.6743),
    ('Houston', 29.7604, -95.3698),
    ('Indianapolis', 39.7684, -86.1581),
    ('Jacksonville', 30.3322, -81.6557),
    ('Las Vegas', 36.1699, -115.1398),
    ('Los Angeles', 34.0522, -118.2437),
    ('Louisville', 38.2527, -85.7585),
    ('Miami/Ft. Lauderdale', 25.7617, -80.1918),
    ('Midsouth', 35.1495, -90.0489),
    ('Nashville', 36.1627, -86.7816),
    ('New Orleans/Mobile', 29.9511, -90.0715),
    ('New York', 40.7128, -74.006),
    ('Northeast', 41.5, -75.0),
    ('Northern New England', 44.0, -71.5),
    ('Orlando', 28.5383, -81.3792),
    ('Philadelphia', 39.9526, -75.1652),
    ('Phoenix/Tucson', 33.4484, -112.074),
    ('Pittsburgh', 40.4406, -79.9959),
    ('Plains', 39.0119, -98.4842),
    ('Portland', 45.5152, -122.6784),
    ('Raleigh/Greensboro', 35.7796, -78.6382),
    ('Richmond/Norfolk', 37.5407, -77.436),
    ('Roanoke', 37.2709, -79.9414),
    ('Sacramento', 38.5816, -121.4944),
    ('San Diego', 32.7157, -117.1611),
    ('San Francisco', 37.7749, -122.4194),
    ('Seattle', 47.6062, -122.3321),
    ('South Carolina', 33.8361, -81.1637),
    ('South Central', 34.7465, -92.2896),
    ('Southeast', 32.3182, -86.9023),
    ('Spokane', 47.6588, -117.426),
    ('St. Louis', 38.627, -90.1994),
    ('Syracuse', 43.0481, -76.1474),
    ('Tampa', 27.9506, -82.4572),
    ('Total U.S.', 39.8283, -98.5795),
    ('West', 39.1178, -120.013),
    ('West Tex/New Mexico', 31.9686, -99.9018)
]


class GeoDistanceEncoder:
    def __init__(self, city_coordinates, reference_points):
        """
        Initialisiert den GeoDistanceEncoder mit den Städten und den Referenzpunkten.

        :param city_coordinates: Dictionary mit Städtenamen und deren (Latitude, Longitude)-Koordinaten
        :param reference_points: Dictionary mit Referenzpunkten und deren (Latitude, Longitude)-Koordinaten
        """
        self.city_coordinates = city_coordinates
        self.reference_points = reference_points

    def fit(self, X, y=None):
        # fit ist nicht notwendig, wird aber für Kompatibilität bereitgestellt.
        return self

    def transform(self, X):
        """
        Berechnet die geografischen Distanzen zu den Referenzpunkten. Falls die Spalten 'latitude' und 'longitude'
        nicht im Datensatz vorhanden sind, werden sie anhand der Stadtzuordnung berechnet.

        :param X: DataFrame mit der Spalte 'geography' (Städte) und optional 'latitude' und 'longitude'
        :return: DataFrame mit den berechneten Distanzen zu den Referenzpunkten
        """
        # Überprüfen, ob die Spalten 'latitude' und 'longitude' bereits vorhanden sind
        if 'latitude' not in X.columns or 'longitude' not in X.columns:
            # Falls die Spalten nicht existieren, die Koordinaten der Städte hinzufügen
            X['latitude'] = X['geography'].map(lambda x: self.city_coordinates.get(x, (None, None))[0])
            X['longitude'] = X['geography'].map(lambda x: self.city_coordinates.get(x, (None, None))[1])

        # Entfernen von Zeilen ohne gültige Koordinaten
        X = X.dropna(subset=['latitude', 'longitude'])

        # Berechnen der Entfernungen zu den Referenzpunkten
        for name, coords in self.reference_points.items():
            X[f'dist_to_{name}'] = X.apply(lambda row: geodesic((row['latitude'], row['longitude']), coords).kilometers, axis=1)

        return X

    def fit_transform(self, X, y=None):
        """
        Führt fit und transform direkt hintereinander aus.

        :param X: DataFrame mit der Spalte 'geography' (Städte) und optional 'latitude' und 'longitude'
        :return: DataFrame mit den berechneten Distanzen zu den Referenzpunkten
        """
        return self.fit(X).transform(X)

# Umwandeln der Liste in ein Dictionary für den Encoder
city_coordinates = {city: (lat, lon) for city, lat, lon in static_coordinates}

# Referenzpunkte (Kalifornien und Mexiko)
reference_points = {
    'california': (36.7783, -119.4179),
    'mexico': (23.6345, -102.5528)
}

# Anwendung des GeoDistanceEncoders
encoder = GeoDistanceEncoder(city_coordinates, reference_points)
df = encoder.fit_transform(df)

# Überprüfen der ersten Zeilen des Datensatzes
print(df[['geography', 'latitude', 'longitude', 'dist_to_california', 'dist_to_mexico']].head())


## East/West Aufteilung

In [ ]:
# Aufteilung in Westküste und Ostküste
def classify_region(dataset):
    west_coast_regions = ['California', 'Portland', 'Seattle', 'Los Angeles', 'San Francisco']
    east_coast_regions = ['New York', 'Boston', 'Miami', 'Philadelphia', 'Baltimore/Washington',
                          'Atlanta', 'Tampa', 'Orlando', 'Hartford/Springfield', 'Raleigh/Greensboro', 'Richmond/Norfolk']
    #closer_east_coast = ['Albany','Buffalo/Rochester','Charlotte','Chicago','Cincinnati/Dayton','Columbus',
    #                     'Dallas/Ft. Worth','Detroit','Grand Rapids','Great Lakes','Harrisburg/Scranton','Houston',
    #                     'Indianapolis','Jacksonville','Louisville','Midsouth','Nashville','New Orleans/Mobile',
    #                  'Northeast','Northern New England','Pittsburgh','Plains','Roanoke','South Carolina',
    #                     'South Central','Southeast','St. Louis','Syracuse','Total U.S']
    #closer_west_coast = ['Boise','Las Vegas','Denver','Phoenix/Tucson','Sacramento','San Diego','Spokane','West Tex/New Mexico','West']

    #merged_east_coast = list(set(east_coast_regions + closer_east_coast))
    #merged_west_coast = list(set(west_coast_regions + closer_west_coast))

    dataset['region'] = dataset['geography'].apply(
    lambda x: 0 if any(west in x for west in west_coast_regions) else
              (1 if any(east in x for east in east_coast_regions) else None)
  )
    return dataset

df = classify_region(df)
df.head(20)
df = df.dropna(subset=['region'])
df

## OneHot Encoder Geography

In [ ]:
def onehot_encode(df, column):
  # Make a copy of the origianl DataFrame
  df = df.copy()

  # Create dummy variables (one-hot encoding)
  dummies = pd.get_dummies(df[column])

  # Convert dummy columns to integers (0 and 1)
  dummies = dummies.astype(int)

  # Concatenate the dummy variables to the original DataFrame
  df = pd.concat([df, dummies], axis=1)

  # Drop the original column from the DataFrame
  df.drop(column, axis =1, inplace = True)
  return df


In [ ]:
df = onehot_encode(df, 'geography')

In [ ]:
# Überprüfung NaN-Werte

# df.isnull().sum()

In [ ]:
df.columns

# Spalten löschen

In [ ]:
# Entfernen der angegebenen Spalten
df = df.drop(columns=['date','geography_encoded', 'latitude', 'longitude'])

# Ausgabe der ersten Zeilen des Datensatzes zur Überprüfung
print(df.head())


In [ ]:
# Entfernen der angegebenen Spalten
df = df.drop(columns=['4046', '4225', '4770','total_bags', 'small_bags', 'large_bags', 'xlarge_bags'])

# Ausgabe der ersten Zeilen des Datensatzes zur Überprüfung
print(df.head())


# Train-Test-Split

In [ ]:
# Annahme: Die Zielvariable, die wir vorhersagen wollen, ist 'average_price'
# Und die Feature-Spalten sind alle anderen Spalten außer 'average_price'

# Splitten in Train- und Test-Datensätze basierend auf dem Jahr
train = df[df['year'] < 2020]  # Training Data (before 2020)
test = df[df['year'] >= 2020]  # Testing Data (2020 and later)

# Features (X) und Zielvariable (y) definieren
X_train = train.drop(columns=['average_price'])
y_train = train['average_price']

X_test = test.drop(columns=['average_price'])
y_test = test['average_price']

# Ausgabe der Formen der Train- und Test-Datensätze
print("Train-Daten: X_train:", X_train.shape, "y_train:", y_train.shape)
print("Test-Daten: X_test:", X_test.shape, "y_test:", y_test.shape)



# Scaler

In [ ]:
# Import necessary libraries
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Assuming 'average_price' is your target variable, and the rest are features
columns_to_scale = ['date_number', 'total_volume', 'dist_to_california', 'dist_to_mexico']  # Exclude 'average_price' from scaling

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit the scaler on the training data only and transform it
X_train[columns_to_scale] = scaler.fit_transform(X_train[columns_to_scale])

# Use the already fitted scaler to transform the test data
X_test[columns_to_scale] = scaler.transform(X_test[columns_to_scale])







In [ ]:
# Entfernen der angegebenen Spalten
X_train = X_train.drop(columns=['year'])
X_test = X_test.drop(columns=['year'])

In [ ]:
#Output the shapes and first few rows of scaled data for verification
X_train.head()


In [ ]:
#Output the shapes and first few rows of scaled data for verification
X_test.head()

# Datenspeicherung in Pickle

In [ ]:
# DataFrame als Pickle-Datei speichern
import pickle

# Annahme: X_train, y_train, X_test, y_test sind bereits definiert

# Speichern in eine Pickle-Datei
with open('data/train_test_data.pkl', 'wb') as f:
    pickle.dump((X_train, y_train, X_test, y_test), f)

print("Die Daten wurden erfolgreich in die Pickle-Datei gespeichert.")


In [ ]:
# # Laden der gespeicherten Pickle-Datei
# with open('data/train_test_data.pkl', 'rb') as f:
#     X_train, y_train, X_test, y_test = pickle.load(f)

# # Überprüfen der geladenen Daten
# print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)
